# 初始化

In [1]:
import os
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers
from keras.models import Model, Sequential
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,add,AveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  

Using TensorFlow backend.


# 读取训练数据

In [2]:
dataset = 'UCM'
base_path='./Dataset/' + dataset
ModelName = "VGG13"
batch_size = 16
epochs = 4

X_Train = np.load(base_path + "/X_Train_224.npy");
Y_Train = np.load(base_path + "/Y_Train.npy");

print("Get %d Train Images" %(len(X_Train)))       
num_classes = Y_Train.shape[1]

Get 26880 Train Images


In [3]:
def VGG13(shape):
    model = Sequential(name=ModelName)  
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=shape,padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Flatten())  
    model.add(Dense(4096,activation='relu'))  
    model.add(Dropout(0.5))  
    model.add(Dense(4096,activation='relu'))  
    model.add(Dropout(0.5))  
    model.add(Dense(num_classes,activation='softmax'))  
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
    return model

In [4]:
model = VGG13(X_Train.shape[1:])
model.summary()

Model: "VGG13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     49280     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168

In [ ]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/4
26880/26880 [==============================] - 169s 6ms/step - loss: nan - accuracy: 0.0475 2s - 
Epoch 2/4
 8128/26880 [========>.....................] - ETA: 1:57 - loss: nan - accuracy: 0.0449

# 保存模型

In [ ]:
import time
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print("Saved Model to disk")

In [ ]:
images = []
labels = []
outputFileName = []

save_folder2 = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder2, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

In [ ]:
if not os.path.exists(save_folder2):
    os.makedirs(save_folder2)
    for Label in All_Labels:        
        os.makedirs(os.path.join(save_folder2, Label))
        
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('dense_2').output)
print("Saved FeatureMap to disk...")
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])    
    np.savetxt(outputFileName[i : i + 1][0], p)
    #file_object = open(outputFileName[i : i + 1][0], 'w')
    #file_object.write(str(p))
    #file_object.close()
    print("\r当前输出：%d" %(i + 1), end= " ")

print("\n保存完成！")

In [ ]:
Count = 0
for i in range(0, len(output)):
    p=model.predict(output[i : i + 1])
    if np.argmax(p)==np.argmax(labels[i : i + 1][0]):
        Count += 1
    
print(Count/len(output))